In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5989468850348270913, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9202108990
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17228484353823979008
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.11.0
dim_ordering: tf


In [5]:
batch_size = 512
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
# print(os.listdir("../input/aptos-multiclass-noaug/aptos_multiclass/"))

In [7]:
train_dir = 'data/train/'
test_dir = 'data/test/'

In [8]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [9]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    validation_split= 0.2,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 319152 images belonging to 6 classes.
Found 79784 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [10]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 319152
nb_validation_samples: 79784
nb_test_samples: 13578

predict_size_train: 624
predict_size_validation: 156
predict_size_test: 27

 num_classes: 6


In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Dense201_Xception_InceptionV3"

In [12]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
# xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [13]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

In [14]:
input_tensor = Input(shape = input_shape)  

base_model1=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model3=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

x3 = base_model3.output
x3 = GlobalAveragePooling2D()(x3)

merge = concatenate([x1, x2, x3])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [15]:
# base_model1.summary()

In [16]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [17]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[572].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[586].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[602].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[636].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[679].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[702].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[722].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[738].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[749].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[783].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[847].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[911].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[975].output
c17 = GlobalAveragePooling2D()(c17) 

c18 = model.layers[1042].output
c18 = GlobalAveragePooling2D()(c18) 

c19 = model.layers[1070].output
c19 = GlobalAveragePooling2D()(c19) 

c20 = model.layers[1106].output
c20 = GlobalAveragePooling2D()(c20) 

c21 = model.layers[1125].output
c21 = GlobalAveragePooling2D()(c21) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, c21])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [21]:
# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [22]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [23]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        'hidden1':hp.choice('hidden1',[4096,2048,1024,512,256]),
        'hidden2':hp.choice('hidden2',[2048,1024,512,256,128]),
        'hidden3':hp.choice('hidden3',[512,256,128,64, 32]),    
        'hidden3':hp.choice('hidden3',[512,256,128,64, 32]),    
        'hidden4':hp.choice('hidden4',[256,128,64, 32, 16]),    
        'hidden5':hp.choice('hidden5',[128,64, 32, 16, 8]),    
    
        'bias_reg': hp.choice('bias_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'act_reg': hp.choice('act_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'ker_reg': hp.choice('ker_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'activation_function':hp.choice('activation_function',["relu","elu","selu","softplus","tanh",])
        }

In [24]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
#     dropout_rate = 0.5
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(params["hidden1"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    model.add(Dropout(0.25))

    model.add(Dense(params["hidden2"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    model.add(Dense(params["hidden3"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))

    model.add(Dense(params["hidden4"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))

    model.add(Dense(params["hidden5"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))

    model.add(Dropout(0.5))
    
    

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [25]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [26]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 800, trials=trials)
print(best)

Parameters testing: 
{'lr': 0.01, 'beta_2': 0.5, 'beta_1': 0.1, 'hidden2': 1024, 'bias_reg': 1e-07, 'activation_function': 'softplus', 'hidden1': 512, 'act_reg': 0.1, 'hidden5': 16, 'hidden3': 32, 'hidden4': 16, 'ker_reg': 1e-05}
Train on 319152 samples, validate on 79784 samples
Epoch 1/1000
 - 24s - loss: 2614.3438 - acc: 0.2283 - val_loss: 435.0336 - val_acc: 0.2312

Epoch 2/1000
 - 19s - loss: 120.0943 - acc: 0.2296 - val_loss: 3.2019 - val_acc: 0.2312

Epoch 3/1000
 - 19s - loss: 1.9273 - acc: 0.2301 - val_loss: 1.6925 - val_acc: 0.2312

Epoch 4/1000
 - 19s - loss: 1.6922 - acc: 0.2295 - val_loss: 1.6926 - val_acc: 0.2312

Epoch 00004: early stopping
  512/79784 [..............................]
 - ETA: 1s

 2560/79784 [..............................]
 - ETA: 2s

 4608/79784 [>.............................]
 - ETA: 2s

 61

ResourceExhaustedError: OOM when allocating tensor with shape[3856,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_81/Adam/gradients/dense_488/weight_regularizer/Square_grad/Mul_1}} = Mul[T=DT_FLOAT, _class=["loc:@training_81/Adam/gradients/AddN_15"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_488/kernel/read, training_81/Adam/gradients/dense_488/weight_regularizer/Square_grad/Mul)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_81/add_14/_25635}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1182_loss_81/add_14", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allo